In [1]:
import sys
sys.path.append('generate_data')
from generator import *
import pandas as pd
pd_v = pd.__version__
python_v = !python -V
print("version", *python_v)
print("version Pandas", pd_v)

version Python 3.10.5
version Pandas 2.0.2


**Входные данные**

In [3]:
!python generate_data/generator.py

In [2]:
df_transactions = read_json_folder(os.path.join(output_location, 'transactions',''))
df_transactions.head()

,customer_id,basket,date_of_purchase
0,C1,"[{'product_id': 'P63', 'price': 1674}]",2023-03-28 00:39:30.397598
1,C4,"[{'product_id': 'P34', 'price': 449}]",2023-03-28 15:22:30.397598
2,C6,"[{'product_id': 'P63', 'price': 401}]",2023-03-28 13:27:30.397598
3,C15,"[{'product_id': 'P01', 'price': 1224}, {'produ...",2023-03-28 06:31:30.397598
4,C28,"[{'product_id': 'P59', 'price': 1264}, {'produ...",2023-03-28 00:28:30.397598


**Решение**

In [3]:
# Распаковка колонки 'basket'
df = df_transactions.explode('basket').reset_index(drop=True)
split_df = pd.DataFrame(df['basket'].to_list(), columns = ['product_id', 'price'])
df = pd.concat([df, split_df], axis=1)
df = df.drop('basket', axis=1)
df.head()

,customer_id,date_of_purchase,product_id,price
0,C1,2023-03-28 00:39:30.397598,P63,1674
1,C4,2023-03-28 15:22:30.397598,P34,449
2,C6,2023-03-28 13:27:30.397598,P63,401
3,C15,2023-03-28 06:31:30.397598,P01,1224
4,C15,2023-03-28 06:31:30.397598,P02,1324


In [4]:
# Перевод даты в день недели
df['date_of_purchase'] = df['date_of_purchase'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S.%f').dayofweek)
df.rename(columns={"date_of_purchase": "day_of_week"}, inplace=True)
df.head()

,customer_id,day_of_week,product_id,price
0,C1,1,P63,1674
1,C4,1,P34,449
2,C6,1,P63,401
3,C15,1,P01,1224
4,C15,1,P02,1324


In [5]:
# Вычисление результирующих колонок
def set_to_list(row):
    return list(set(row))

df['quantity'] = 1
df = df.groupby(['customer_id', 'day_of_week']).agg(
    price_max=pd.NamedAgg(column='price', aggfunc="max"),
    price_min=pd.NamedAgg(column='price', aggfunc="min"),
    price_median=pd.NamedAgg(column='price', aggfunc="median"),
    price_mean=pd.NamedAgg(column='price', aggfunc="mean"),
    quantity=pd.NamedAgg(column='quantity', aggfunc="sum"),
    products=pd.NamedAgg(column='product_id', aggfunc=set_to_list)
    ).reset_index()


In [6]:
# Результат
df

,customer_id,day_of_week,price_max,price_min,price_median,price_mean,quantity,products
0,C1,0,1946,395,1541.0,1300.800000,5,"[P48, P63, P43, P61, P52]"
1,C1,1,1674,357,811.0,987.200000,5,"[P59, P45, P63, P58, P01]"
2,C1,3,1363,895,1224.0,1160.666667,3,"[P13, P15, P17]"
3,C1,4,1870,906,1388.0,1388.000000,2,"[P63, P64]"
4,C1,5,831,778,804.5,804.500000,2,"[P43, P57]"
...,...,...,...,...,...,...,...,...
807,C99,0,1075,441,499.0,671.666667,3,"[P59, P41]"
808,C99,1,1563,1004,1283.5,1283.500000,2,"[P23, P53]"
809,C99,4,226,124,175.0,175.000000,2,"[P55, P57]"
810,C99,5,1735,1001,1316.0,1350.666667,3,"[P43, P54, P51]"
